In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

import matplotlib.pyplot as plt

from bayes_opt import BayesianOptimization

In [2]:
# Load data set and target values
data, target = make_classification(
    n_samples=1000,
    n_features=45,
    n_informative=12,
    n_redundant=7)

In [3]:
def svccv(C, gamma):
    val = cross_val_score(
        SVC(C=C, gamma=gamma, random_state=2),
        X=data, y=target, scoring='f1', cv=2
    ).mean()

    return val

def rfccv(n_estimators, min_samples_split, max_features):
    val = cross_val_score(
        RFC(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999),
            random_state=2
        ),
        X=data, y=target, scoring='f1', cv=2
    ).mean()
    return val

In [4]:
svcBO = BayesianOptimization(svccv,
    {'C': (0.001, 100), 'gamma': (0.0001, 0.1)})

rfcBO = BayesianOptimization(
    rfccv,
    {'n_estimators': (10, 250),
    'min_samples_split': (2, 25),
    'max_features': (0.1, 0.999)}
)

svcBO.maximize(init_points=5, n_iter=10)
print('-' * 53)
rfcBO.maximize(init_points=5, n_iter=10)

print('-' * 53)
print('Final Results')
print('SVC: %f' % svcBO.max['target'])
print('RFC: %f' % rfcBO.max['target'])

print('-' * 53)
print('Best Parameters')
print(f'SVC: {svcBO.max["params"]}')
print(f'RFC: {rfcBO.max["params"]}')

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        |  0.9232   |  50.08    |  0.01547  |
|  2        |  0.669    |  36.3     |  0.05698  |
|  3        |  0.016    |  22.34    |  0.09917  |
|  4        |  0.03556  |  61.35    |  0.09349  |
|  5        |  0.9253   |  20.91    |  0.01418  |
|  6        |  0.7461   |  94.94    |  0.05181  |
|  7        |  0.9232   |  50.09    |  0.01591  |
|  8        |  0.846    |  17.45    |  0.0001   |
|  9        |  0.01202  |  44.15    |  0.1      |
|  10       |  0.4953   |  89.12    |  0.06598  |
|  11       |  0.8623   |  100.0    |  0.0001   |
|  12       |  0.9178   |  8.043    |  0.008106 |
|  13       |  0.8519   |  2.945    |  0.000312 |
|  14       |  0.8614   |  76.16    |  0.0001   |
|  15       |  0.01202  |  70.74    |  0.1      |
-----------------------------------------------------
|   iter    |  target   | max_fe... | min_sa... | n_esti... |
----------------------------------